# Secret Box

The `secretbox` API uses secret key authenticated encryption to encrypt and authenticate a message with a secret key that all parties must share.

[Libsodium Documentation](https://doc.libsodium.org/secret-key_cryptography/secretbox)

Functions:
```
    crypto_secretbox_keygen() -> bytea

    crypto_secretbox_noncegen() -> bytea

    crypto_secretbox(message bytea, nonce bytea, key bytea) -> bytea

    crypto_secretbox(message bytea, nonce bytea, key_id bigint, context bytea = 'pgsodium') -> bytea

    crypto_secretbox_open(ciphertext bytea, nonce bytea, key bytea) -> bytea

    crypto_secretbox_open(ciphertext bytea, nonce bytea, key_id bigint, context bytea = 'pgsodium') -> bytea
```

`crypto_secretbox_keygen()` generates a random secret key which can be
used to encrypt and decrypt messages. The role `pgsodium_keymaker` is required to call this function.

`crypto_secretbox_noncegen()` generates a random nonce which will be
used when encrypting messages.  For security, each nonce must be used
only once, though it is not a secret.  The purpose of the nonce is to
add randomness to the message so that the same message encrypted
multiple times with the same key will produce different ciphertexts.  The role `pgsodium_keyiduser` or greater is required to call this function.

`crypto_secretbox()` encrypts a message using a previously generated
nonce and secret key.  The encrypted message can be decrypted using
`crypto_secretbox_open()`  Note that in order to decrypt the message,
the original nonce will be needed. The role `pgsodium_keyholder` is required to call the raw `key bytea` versions of these functions.  The key id versions of the functions can be called with the role `pgsodium_keyiduser`.



In [4]:
%load_ext sql
%sql postgresql://postgres@/

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


Encryption requires a key and a nonce.  The nonce doesn't have to be confidential, but it should never ever be reused with the same key. The easiest way to generate a nonce is to use `crypto_secretbox_noncegen`:

In [ ]:
key = %sql select pgsodium.crypto_secretbox_keygen()::text
key = key[0][0]

nonce = %sql select pgsodium.crypto_secretbox_noncegen()::text
nonce = nonce[0][0]

## Encryption

A new secretbox is created with the key and the nonce:

In [18]:
secretbox = %sql SELECT crypto_secretbox::text from pgsodium.crypto_secretbox('bob is your uncle', :nonce, (:key)::bytea)
secretbox = secretbox[0][0]
print('The encrypted secretbox is: ', secretbox)

 * postgresql://postgres@/
1 rows affected.
 * postgresql://postgres@/
1 rows affected.
 * postgresql://postgres@/
1 rows affected.
The encrypted secretbox is:  \x7b11d8e3659f6fe2a7762f082019c607d5d64fd5f805f6ff6df68266664a6ec335


## Decryption

Decryption requires the same key and nonce.

In [19]:
plaintext = %sql SELECT crypto_secretbox_open FROM pgsodium.crypto_secretbox_open(:secretbox, :nonce, (:key)::bytea)
print('The decrypted message is :', plaintext[0][0].tobytes().decode('utf8'))

 * postgresql://postgres@/
1 rows affected.
The decrypted message is : bob is your uncle
